# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [4]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset 

In [5]:
# Dataset name
dataset_name = "facebook/ExploreToM"

In [6]:
# TODO: load and explore the dataset
ds = load_dataset(dataset_name)

In [11]:
# TODO: number of rows in the dataset
print(ds.shape)

# TODO: Keys in the dataset
print(ds.keys())

# TODO: Feature names
print(ds['train'].features)

# TODO: Display a single row
idx = 1
for k, v in ds['train'][idx].items():
   print(f'key = {k}, value = {v}')


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value('string'), 'infilled_story': Value('string'), 'question': Value('string'), 'expected_answer': Value('string'), 'qprop=params': Value('string'), 'qprop=nth_order': Value('int64'), 'qprop=non_unique_mental_state': Value('bool'), 'sprop=is_false_belief_story_1st': Value('bool'), 'sprop=is_false_belief_story_1st_and_2nd': Value('bool'), 'sprop=story_accuracy_1st_raw': Value('float64'), 'sprop=story_accuracy_1st_infilled': Value('float64'), 'sprop=global_idx': Value('int64'), 'param=story_type': Value('string'), 'param=num_stories_total': Value('int64'), 'param=max_sentences': Value('int64'), 'param=num_people': Value('int64'), 'param=num_moves': Value('int64'), 'param=num_rooms': Value('int64')}
key = story_structure, value = Kaylee entered the hotel lobby. Kaylee moved the silver letter opener to the wooden desk drawer, which is also located in the hotel lobby. While this action was happening, Liam witnessed this action

In [12]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [14]:
# TODO: Summarise the text with the pipeline's default model

qna = pipeline('question-answering')

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [18]:
idx = 10
question = ds['train'][idx]['question']
story = ds['train'][idx]['story_structure']
#story = ds['train'][idx]['infilled_story']
answer = ds['train'][idx]['expected_answer']

predict_answer = qna(question=question, context=story)

print('Predition: ', predict_answer)
print('Actual: ', answer)

Predition:  {'score': 0.683438777923584, 'start': 286, 'end': 301, 'answer': 'leather satchel'}
Actual:  wooden chest


## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [19]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [20]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [21]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [23]:
# TODO: Encode text
text = 'hello world'
text = 'Thou shalt not make a machine in the likeness of the human mind.'

# return tokens as Pytorch Tensors
enc_text = tokenizer(text, return_tensors='pt')

print(enc_text)

{'input_ids': tensor([[  101,   157, 14640,   188, 24537,  1136,  1294,   170,  3395,  1107,
          1103,  1176,  1757,  1104,  1103,  1769,  1713,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [24]:
dec_text = tokenizer.decode(enc_text.input_ids[0])
print(dec_text)

[CLS] Thou shalt not make a machine in the likeness of the human mind. [SEP]


In [25]:
for tok in enc_text.input_ids[0]:
   dec_tok = tokenizer.decode(tok)
   print(tok, dec_tok)

tensor(101) [CLS]
tensor(157) T
tensor(14640) ##hou
tensor(188) s
tensor(24537) ##halt
tensor(1136) not
tensor(1294) make
tensor(170) a
tensor(3395) machine
tensor(1107) in
tensor(1103) the
tensor(1176) like
tensor(1757) ##ness
tensor(1104) of
tensor(1103) the
tensor(1769) human
tensor(1713) mind
tensor(119) .
tensor(102) [SEP]


In [28]:
# TODO: Encoding multiple texts
texts = [
   "Big black bug bleeds black blood",
   "t is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife"
]

enc_texts = tokenizer(texts, return_tensors='pt', padding=True)

print(enc_texts.input_ids)
print(enc_texts.attention_mask)


tensor([[  101,  2562,  1602, 15430, 24752,  1116,  1602,  1892,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,   189,  1110,   170,  3062, 23578,  8646,   117,  1115,   170,
          1423,  1299,  1107,  6224,  1104,   170,  1363,  8977,   117,  1538,
          1129,  1107,  1328,  1104,   170,  1676,   102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])


In [ ]:
# TODO: Decode text


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [29]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering

In [30]:
# TODO: Load question answer model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


In [47]:
# TODO: Encode context and question
idx = 100
question = ds['train'][idx]['question']
story = ds['train'][idx]['story_structure']
story = ds['train'][idx]['infilled_story']
answer = ds['train'][idx]['expected_answer']

enc_question = tokenizer(question, return_tensors='pt')
enc_story = tokenizer(story, return_tensors='pt')

enc_input = tokenizer(question, story, return_tensors='pt', padding=True)
print(enc_input)

{'input_ids': tensor([[  101,  1327,  1674, 10206,  1341,  1164, 23065,   112,   188,  6369,
          1113,  3782,  6213, 10700,   136,   113,  3520,  1164,  1122,   120,
          1674,  1136,  1221,  1164,  1122,   114,   102,  1109,  3258,  8656,
          1104,  5101,  4204, 15888,  1103, 18652,  5769,  1104,  1103,  3782,
          4745,   117,  9616,   170, 21088,  6814,  1166, 10389,  1104, 12960,
          1116,  1105, 14312,   119,   138,  7859,  5974,  1104,  3618, 13433,
          1105,  6656, 20049, 20619,  1194,  1103,  1586,   117, 11241, 10134,
          1114,  1103,  6531,  3807,  1104,  7053,  1105,  1390,   117,  1112,
          1103,  1480,   112,   188, 24039,  1125,  1198,  4972,  1106,  8362,
         10787,   119,  8521,  5002, 25649,  1154,  1103,  1514,  9459,   117,
          1103,  3807,  1104,  1103,  3782,  1796, 17429, 17032,  1118,  1103,
         11246,  2928,   119,  8521,  4144,  1763,  1103,  5134, 11246, 23841,
          1116,   117, 17862,  1114,  

In [43]:
print(tokenizer.decode(enc_input.input_ids[0]))

[CLS] In which container was the pocket watch at the beginning? [SEP] The bustling theater was a hive of activity on this chilly autumn evening, its worn wooden floors and faded velvet curtains a testament to years of countless performances. The dimly lit green room, tucked away backstage, was a cozy refuge from the chaos, its plush armchairs and ornate wooden chest offering a warm respite for those who needed it. Dylan slipped away from the crowd, disappearing behind a tattered curtain as he made his way to a more secluded space. The soft glow of a floor lamp in the green room enveloped him, providing a sense of tranquility amidst the pre - show chaos. Dylan carefully placed the pocket watch in the ornate wooden chest, hidden from view, and Clayton caught a glimpse of this sneaky maneuver from his secret vantage point. Clayton ' s eyes narrowed as he pushed open the creaky door and stepped into the green room, the sudden movement making the softly lit space seem almost anticipatory. I

In [48]:
result = model(enc_input.input_ids, enc_input.attention_mask)

In [36]:
print(result)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -3.3331,  -7.0589,  -7.4497,  -6.9617,  -8.8258,  -8.2602,  -7.9495,
          -8.8655,  -8.0126,  -8.1991,  -7.5855,  -5.1998,  -7.5699,  -3.9040,
          -6.8398,  -6.2395,  -4.4169,  -7.9310,  -7.0676,  -3.1802,  -5.1120,
          -4.8768,  -4.3839,  -6.6323,  -3.5747,   2.9140,   3.6676,   1.6920,
          -7.0250,  -7.1047,  -8.1779,  -8.5559,  -7.2667,  -6.7811,  -5.9012,
          -4.7313,  -7.8112,  -7.6731,  -6.8141,  -7.7848,  -8.1972,  -9.7429,
          -9.0243,  -8.9493,  -4.6271,  -8.0457,  -7.6556,  -9.0675,  -7.7875,
          -6.2783,  -8.4216, -10.1118,  -8.9245,  -7.9942,  -8.5808,  -9.3803,
          -7.4695,  -4.4636,  -7.2955,  -6.1989,  -4.6764,  -7.6352,  -5.7930,
          -1.5447,  -3.4093,  -1.6110,  -2.4669,  -5.1233,  -1.5189,   5.8467,
           7.0104,   4.0358,  -0.2083,  -5.4350,  -5.8350,  -6.3616,  -7.2949,
          -5.6393,  -4.6958,  -3.9456,  -3.6151,  -6.5309,  -6.0961,  -7.5699]

In [49]:
# TODO: Tokenize the inputs
start_ans = torch.argmax(result.start_logits)
end_ans = torch.argmax(result.end_logits) + 1
enc_answer = enc_input.input_ids[0][start_ans: end_ans]

print(enc_answer)
predict_answer = tokenizer.decode(enc_answer)
print(predict_answer)
print(answer)



tensor([10206, 22546,   170,  3802])
Isabella mouthed a message
does not know about it


In [34]:
# Ensure minimum and maximum token length in the answer
import torch
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [ ]:
# TODO Return a minimum of 5 tokens


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many stories has Dickens wrote?"

